In [4]:
from slide_tiling.tiling import save_tile_images
from pathlib import Path

DATA = "C:\\Users\\MS\\Desktop\\IDEAS\\Projekty\\CD34\\Dane\\CD34_adnotacje_final"
OUT = "C:\\Users\\MS\\Desktop\\IDEAS\\Projekty\\CD34\\Dane\\CD34_tiles"

all_wsi = [
    subdir_name.name for subdir_name in Path(DATA).iterdir() if subdir_name.is_dir()
]

test_wsi = [
    "22_19269A1_7",
    "23_08703A1_7",
    "23_11101A1_7",
]

excluded = set("23_24187A1_7")

train_wsi = [wsi for wsi in all_wsi if wsi not in set(test_wsi) and wsi not in excluded]

print(train_wsi)

['22_09617A1_7', '22_16541A1_7', '22_21450A1_7', '22_23859A1_7', '23_00376A1_7', '23_08116A1_7', '23_09909A1_7', '23_13438A1_7', '23_17699A1_7']


In [3]:
from slide_tiling.tiling import save_tile_images
from pathlib import Path

DATA = "C:\\Users\\MS\\Desktop\\IDEAS\\Projekty\\CD34\\Dane\\CD34_adnotacje_final"
OUT = "C:\\Users\\MS\\Desktop\\IDEAS\\Projekty\\CD34\\Dane\\CD34_tiles"

all_wsi = [
    subdir_name.name for subdir_name in Path(DATA).iterdir() if subdir_name.is_dir()
]

test_wsi = [
    "22_19269A1_7",
    "23_08703A1_7",
    "23_11101A1_7",
]

excluded = set("23_24187A1_7")

train_wsi = [wsi for wsi in all_wsi if wsi not in set(test_wsi) and wsi not in excluded]

selected_classes = ["Mieloblasty", "Megakariocyty", "komorki"]

print(len(all_wsi), "WSIs: ", all_wsi)
print(len(train_wsi), "Train WSIs: ", train_wsi)
print(len(test_wsi),  "Test WSIs: ", test_wsi)

Path(OUT).mkdir(parents=True, exist_ok=True)
save_tile_images(
    train_wsi_names=train_wsi,
    test_wsi_names=test_wsi,
    data_dir=DATA,
    out_dir=OUT,
    selected_classes=selected_classes,
    bbox_sizes={"Mieloblasty": (48, 48), "Megakariocyty": (68, 68), "komorki": (48, 48)},
    class_shortcodes={"Mieloblasty": "mb", "Megakariocyty": "mc", "komorki": "k"},
    tile_size=None,
    show_n=None,
    )

12 WSIs:  ['22_09617A1_7', '22_16541A1_7', '22_19269A1_7', '22_21450A1_7', '22_23859A1_7', '23_00376A1_7', '23_08116A1_7', '23_08703A1_7', '23_09909A1_7', '23_11101A1_7', '23_13438A1_7', '23_17699A1_7']
9 Train WSIs:  ['22_09617A1_7', '22_16541A1_7', '22_21450A1_7', '22_23859A1_7', '23_00376A1_7', '23_08116A1_7', '23_09909A1_7', '23_13438A1_7', '23_17699A1_7']
3 Test WSIs:  ['22_19269A1_7', '23_08703A1_7', '23_11101A1_7']
46
[get_images_for_group] Mieloblasty:  46
12
[get_images_for_group] Megakariocyty:  12
965
[get_images_for_group] komorki:  965
[get_tile_images_from_wsi] WSI images:  1023
ROIs: 1023
IMGs: 1023
CLASSes: 1023
22_19269A1_7 (test): 1023 roi_id, 1023 images
1659
[get_images_for_group] Mieloblasty:  1659
1
[get_images_for_group] Megakariocyty:  1
1371
[get_images_for_group] komorki:  1371
[get_tile_images_from_wsi] WSI images:  3031
ROIs: 3031
IMGs: 3031
CLASSes: 3031
23_08703A1_7 (test): 3031 roi_id, 3031 images
587
[get_images_for_group] Mieloblasty:  587
9
[get_images

In [19]:
import xml.etree.ElementTree as ET
from pathlib import Path
from collections import Counter

DATA = "C:\\Users\\MS\\Desktop\\IDEAS\\Projekty\\CD34\\Dane\\CD34_adnotacje_final"
all_wsi = [
    # subdir_name.name
    # for subdir_name in Path(DATA).iterdir()
    # if subdir_name.is_dir()
    "23_24187A1_7"
]
print("Liczba WSI: ", len(all_wsi))

# DATA = "..\\data\\xmls"
# all_xmls = [xml_file for xml_file in Path(DATA).iterdir()]

n_xml           = Counter()
n_megakariocyty = Counter()
n_mieloblasty   = Counter()
n_komorki       = Counter()
n_inne          = Counter()
inne_nazwy = set()

dot_subgroups = ["Mieloblasty", "Megakariocyty", "Artefakty", "komorki"]

global_types = Counter()
global_dot_subgroups = Counter()

for i, wsi in enumerate(all_wsi):

    xml_path = Path(DATA, f"{wsi}.xml")
    # for xml_filename in all_xmls:
    #     xml_path = Path(xml_filename)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    annotations = root.findall(".//Annotation")
    if len(annotations) > 0:
        n_xml['files'] += 1

    type_counter = Counter()
    dot_group_counter = Counter()

    for annotation in annotations:

        if annotation.attrib.get("PartOfGroup") == "Megakariocyty" and not annotation.attrib.get("Type") == "Dot":
            print(
                "WSI: ", wsi, "\n",
                "Type: ",
                annotation.attrib.get("Type"), "\n",
                "Group: ",
                annotation.attrib.get("PartOfGroup"),
            )

        n_xml[annotation.attrib["PartOfGroup"]] += 1

        annotation_type = annotation.attrib.get("Type")
        part_of_group = annotation.attrib.get("PartOfGroup", "Nieznane")
        if annotation_type is None:
            continue
        type_counter[annotation_type] += 1
        global_types[annotation_type] += 1
        if annotation_type in ("Dot", "Spline"):
            dot_group_counter[part_of_group] += 1
            global_dot_subgroups[part_of_group] += 1

print("Liczba plików xml: ", n_xml.get('files'))
print("Liczba megakariocytów: ", n_xml.get("Megakariocyty"))
print("Liczba mieloblastów: ", n_xml.get("Mieloblasty"))
print("Liczba aretfaktów: ", n_xml.get("Artefakty"))
print("Liczba komórek: ", n_xml.get("komorki"))
print("-"*12)
print("Rectangle (Obszary):",  global_types.get("Rectangle", 0))
print("Polygon (Naczynia): ", global_types.get("Polygon", 0))
print("Dot (Komórki): ", global_types.get("Dot", 0))
for subgroup in dot_subgroups:
    print(f"Dot - {subgroup}: ", global_dot_subgroups.get(subgroup, 0))

Liczba WSI:  1
Liczba plików xml:  1
Liczba megakariocytów:  1027
Liczba mieloblastów:  2408
Liczba aretfaktów:  53
Liczba komórek:  None
------------
Rectangle (Obszary): 10
Polygon (Naczynia):  80
Dot (Komórki):  3488
Dot - Mieloblasty:  2408
Dot - Megakariocyty:  1027
Dot - Artefakty:  53
Dot - komorki:  0


In [3]:
import os
TRAIN = "C:\\Users\\MS\\Desktop\\IDEAS\\Projekty\\CD34\\Dane\\CD34_tiles\\train"
TEST  = "C:\\Users\\MS\\Desktop\\IDEAS\\Projekty\\CD34\\Dane\\CD34_tiles\\test"

for class_name in ["Megakariocyty", "Mieloblasty", "Komorki"]:
    test_dir = Path(TEST, class_name)
    train_dir = Path(TRAIN, class_name)
    
    n_test = len(os.listdir(test_dir))
    n_train = len(os.listdir(train_dir))
    
    print(f"{class_name}: {n_train} train images, {n_test} test images")

Megakariocyty: 13 train images, 22 test images
Mieloblasty: 8890 train images, 2292 test images
Komorki: 4880 train images, 965 test images
